# Part 2, Topic 1, Lab A: Instruction Power Differences (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *Now that you've been introduced to the ChipWhisperer platform, we'll be using it to learn more about how the power consumed by a microcontroller varies based on what operations it is performing and what instructions it is executing.*

**LEARNING OUTCOMES:**

* Capturing a power trace with ChipWhisperer
* Making connections between various simple instructions and power traces

## Prerequisites

Hold up! Before you continue, check you've done the following tutorials:

* ☑ Jupyter Notebook Intro (you should be OK with plotting & running blocks).
* ☑ SCA101 Intro (you should have an idea of how to get hardware-specific versions running).

## Setup

At this point you've got to insert code to perform the power trace capture. There are two options here:
* Capture from physical device.
* Read from a file.

You get to choose your adventure - see the two notebooks with the same name of this, but called `(SIMULATED)` or `(HARDWARE)` to continue. Inside those notebooks you should get some code to copy into the following section, which will define the capture function.

Be sure you get the `"✔️ OK to continue!"` print once you run the cell afterwards, otherwise things will fail later on!

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWHUSKY'

In [2]:
%%bash
cd ../../../hardware/victims/firmware/
mkdir -p simpleserial-base-lab2 && cp -r simpleserial-base/* $_
cd simpleserial-base-lab2

In [3]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=NONE

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (Arm GNU Toolchain 12.3.Rel1 (Build arm-12.35)) 12.3.1 20230626
Copyright (C) 2022 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-base.c ...
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/startup_sam4s.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/sam4s_hal.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/uart.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/pio.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/system_sam4s.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/sysclk.c ...
-e Do

/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-closer.o): in function `_close_r':
closer.c:(.text._close_r+0xc): warning: _close is not implemented and will always fail
/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-lseekr.o): in function `_lseek_r':
lseekr.c:(.text._lseek_r+0x10): warning: _lseek is not implemented and will always fail
/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/ar

-e Done!
.
Creating load file for Flash: simpleserial-base-CWHUSKY.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.hex
.
Creating load file for Flash: simpleserial-base-CWHUSKY.bin
arm-none-eabi-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.bin
.
Creating load file for EEPROM: simpleserial-base-CWHUSKY.eep
arm-none-eabi-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .eeprom=0 --no-change-warnings -O ihex simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.eep || exit 0
.
Creating Extended Listing: simpleserial-base-CWHUSKY.lss
arm-none-eabi-objdump -h -S -z simpleserial-base-CWHUSKY.elf > simpleserial-base-CWHUSKY.lss
.
Creating Symbol Table: simpleserial-base-CWHUSKY.sym
arm-none-eabi-nm -n simpleserial-base-CWHUSKY.elf > simpleserial-base-CWHUSKY.sym
SS_VER set to SS_VER_1_1
SS_VER set to SS

In [4]:
import chipwhisperer as cw
try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()
   
try:
    target = cw.target(scope)
except IOError:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope)

print("INFO: Found ChipWhisperer😍")

if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
    prog = cw.programmers.SAM4SProgrammer
else:
    prog = None
    
import time
time.sleep(0.05)
scope.default_setup()

if PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
    scope.io.target_pwr = 0
    time.sleep(0.2)
    scope.io.target_pwr = 1
    time.sleep(0.2)
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

INFO: Found ChipWhisperer😍


In [5]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-base-lab2/simpleserial-base-{}.hex".format(PLATFORM))

In [6]:
def capture_trace(_ignored=None):
    ktp = cw.ktp.Basic()
    key, text = ktp.next()
    return cw.capture_trace(scope, target, text).wave

In [7]:
wave = capture_trace()
print("✔️ OK to continue!")

✔️ OK to continue!


## "Empty" Trace

In [3]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=NONE

In [16]:
wave = capture_trace()

In [10]:
cw.plot(wave)

:Curve   [x]   (y)

## 20 multiples

```C
	volatile long int A = 0x2BAA;
	A *= 2;
	A *= 2;
	A *= 2;
	A *= 2;
	A *= 2;
	
	A *= 2;
	A *= 2;
	A *= 2;
	A *= 2;
	A *= 2;

	A *= 2;
	A *= 2;
	A *= 2;
	A *= 2;
	A *= 2;
	
	A *= 2;
	A *= 2;
	A *= 2;
	A *= 2;
	A *= 2;
```

In [11]:
%%bash -s "$PLATFORM"
pwd
cd ../../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=NONE

/Users/rose/cat/chipwhisperer/chipwhisperer-jupyter/user/sca101
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (Arm GNU Toolchain 12.3.Rel1 (Build arm-12.35)) 12.3.1 20230626
Copyright (C) 2022 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-base.c ...
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/startup_sam4s.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/sam4s_hal.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/uart.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/pio.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/system_sam4s.c ...

/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-closer.o): in function `_close_r':
closer.c:(.text._close_r+0xc): warning: _close is not implemented and will always fail
/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-lseekr.o): in function `_lseek_r':
lseekr.c:(.text._lseek_r+0x10): warning: _lseek is not implemented and will always fail
/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/ar

-e Done!
.
Creating load file for Flash: simpleserial-base-CWHUSKY.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.hex
.
Creating load file for Flash: simpleserial-base-CWHUSKY.bin
arm-none-eabi-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.bin
.
Creating load file for EEPROM: simpleserial-base-CWHUSKY.eep
arm-none-eabi-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .eeprom=0 --no-change-warnings -O ihex simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.eep || exit 0
.
Creating Extended Listing: simpleserial-base-CWHUSKY.lss
arm-none-eabi-objdump -h -S -z simpleserial-base-CWHUSKY.elf > simpleserial-base-CWHUSKY.lss
.
Creating Symbol Table: simpleserial-base-CWHUSKY.sym
arm-none-eabi-nm -n simpleserial-base-CWHUSKY.elf > simpleserial-base-CWHUSKY.sym
SS_VER set to SS_VER_1_1
SS_VER set to SS

In [12]:
wave2 = capture_trace()
# blue is empty trace, red is the 20 multiples
cw.plot(wave) * cw.plot(wave2)

:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

## for loop for 20 multiples
```C
volatile long int A = 0x2BAA;
    for(volatile int i = 0; i < 20; i++){
        A = A * 2;
    }
```

In [14]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=NONE

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (Arm GNU Toolchain 12.3.Rel1 (Build arm-12.35)) 12.3.1 20230626
Copyright (C) 2022 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-base.c ...
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/startup_sam4s.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/sam4s_hal.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/uart.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/pio.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/system_sam4s.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/sysclk.c ...
-e Do

/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-closer.o): in function `_close_r':
closer.c:(.text._close_r+0xc): warning: _close is not implemented and will always fail
/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-lseekr.o): in function `_lseek_r':
lseekr.c:(.text._lseek_r+0x10): warning: _lseek is not implemented and will always fail
/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/ar

-e Done!
.
Creating load file for Flash: simpleserial-base-CWHUSKY.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.hex
.
Creating load file for Flash: simpleserial-base-CWHUSKY.bin
arm-none-eabi-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.bin
.
Creating load file for EEPROM: simpleserial-base-CWHUSKY.eep
arm-none-eabi-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .eeprom=0 --no-change-warnings -O ihex simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.eep || exit 0
.
Creating Extended Listing: simpleserial-base-CWHUSKY.lss
arm-none-eabi-objdump -h -S -z simpleserial-base-CWHUSKY.elf > simpleserial-base-CWHUSKY.lss
.
Creating Symbol Table: simpleserial-base-CWHUSKY.sym
arm-none-eabi-nm -n simpleserial-base-CWHUSKY.elf > simpleserial-base-CWHUSKY.sym
SS_VER set to SS_VER_1_1
SS_VER set to SS

In [16]:
wave3 = capture_trace()
# blue is empty, red is 20 multiples, tan is 20 multiples in a for loop 
cw.plot(wave) * cw.plot(wave2) *cw.plot(wave3)

:Overlay
   .Curve.I   :Curve   [x]   (y)
   .Curve.II  :Curve   [x]   (y)
   .Curve.III :Curve   [x]   (y)

## divide in for loop

volatile long int A = 0x2BAA;
    for(volatile int i = 0; i < 20; i++){
        A = A * 2;
    }

In [17]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=NONE

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (Arm GNU Toolchain 12.3.Rel1 (Build arm-12.35)) 12.3.1 20230626
Copyright (C) 2022 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-base.c ...
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/startup_sam4s.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/sam4s_hal.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/uart.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/pio.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/system_sam4s.c ...
-e Done!
.
Compiling:
-en     .././hal/sam4s/sysclk.c ...
-e Do

/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-closer.o): in function `_close_r':
closer.c:(.text._close_r+0xc): warning: _close is not implemented and will always fail
/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-lseekr.o): in function `_lseek_r':
lseekr.c:(.text._lseek_r+0x10): warning: _lseek is not implemented and will always fail
/Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/arm-none-eabi/12.3.1/../../../../arm-none-eabi/bin/ld: /Applications/ArmGNUToolchain/12.3.rel1/arm-none-eabi/bin/../lib/gcc/ar

-e Done!
.
Creating load file for Flash: simpleserial-base-CWHUSKY.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.hex
.
Creating load file for Flash: simpleserial-base-CWHUSKY.bin
arm-none-eabi-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.bin
.
Creating load file for EEPROM: simpleserial-base-CWHUSKY.eep
arm-none-eabi-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .eeprom=0 --no-change-warnings -O ihex simpleserial-base-CWHUSKY.elf simpleserial-base-CWHUSKY.eep || exit 0
.
Creating Extended Listing: simpleserial-base-CWHUSKY.lss
arm-none-eabi-objdump -h -S -z simpleserial-base-CWHUSKY.elf > simpleserial-base-CWHUSKY.lss
.
Creating Symbol Table: simpleserial-base-CWHUSKY.sym
arm-none-eabi-nm -n simpleserial-base-CWHUSKY.elf > simpleserial-base-CWHUSKY.sym
SS_VER set to SS_VER_1_1
SS_VER set to SS

In [ ]:
## divide for loop (blue) vs multiplication for loop (red)

In [20]:
wave4 = capture_trace()
cw.plot(wave4) * cw.plot(wave3)


:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

In [35]:
cw.plot(wave) * cw.plot(wave2)* cw.plot(wave3)*cw.plot(wave4)
# empty trace= blue, 20 multiples = red, 20 multiples for loop = tan, 20 divides 

:Overlay
   .Curve.I   :Curve   [x]   (y)
   .Curve.II  :Curve   [x]   (y)
   .Curve.III :Curve   [x]   (y)
   .Curve.IV  :Curve   [x]   (y)

You can probably see a similar pattern to the faster instruction, but each go through the loop takes longer. We would also expect that longer instructions would also consume more power. Can you see this in your plot?

**HINT: In order to measure current, ChipWhisperer measures voltage drop across a shunt resistor. This means that the power trace is actually inverted (aka large negative swings are areas of higher power consumption).**

## Conclusions & Next Steps

By now you should be reasonably convinced that we can gain some understanding of what a microcontroller is doing by looking at its power consumption. Going forward, we'll be focusing on more objective measurements, but what we did in this lab is still very valuable. AES implementations, for example, will often have a very distinct shape. You can use this to help identify what an unknown device is doing at different times.

Of course, there's a lot more you can do with this lab. For example, does an add look any different from an XOR? Next time, we'll be using what we learned in this lab to break a simple password check.

---
<small>NO-FUN DISCLAIMER: This material is Copyright (C) NewAE Technology Inc., 2015-2020. ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

Tutorials derived from our open-source work must be released under the associated open-source license, and notice of the source must be *clearly displayed*. Only original copyright holders may license or authorize other distribution - while NewAE Technology Inc. holds the copyright for many tutorials, the github repository includes community contributions which we cannot license under special terms and **must** be maintained as an open-source release. Please contact us for special permissions (where possible).

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</small>

In [36]:
scope.dis()
target.dis()